## Extract Table from PDF

In [65]:
import camelot
import pdfplumber
import pandas as pd

def extract_from_pdf(pdf_path: str) -> pd.DataFrame:
    try:
        # First try Camelot (works well for structured tables)
        tables = camelot.read_pdf(pdf_path, pages="all", strip_text="\n")
        if tables and len(tables) > 0:
            df = tables[0].df
            df = df.rename(columns=df.iloc[0]).drop(df.index[0]).reset_index(drop=True)
            return df
    except Exception as e:
        print("Camelot failed:", e)

    # Fallback → pdfplumber
    rows = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                rows.extend(table)

    if not rows:
        raise ValueError("No tables found in PDF using Camelot or pdfplumber")

    df = pd.DataFrame(rows[1:], columns=rows[0])  # first row = headers
    return df



## Normalize Function

In [ ]:
# import pandas as pd

# def normalize_bank_statement(df: pd.DataFrame, bank: str) -> pd.DataFrame:
#     bank = bank.lower().strip()
    
#     if bank == "hdfc":
#         mapping = {
#             "Date": "Date",
#             "Narration": "Description",
#             "Withdrawal": "Debit",
#             "Deposit": "Credit",
#             "ClosingBalance": "Balance"
#         }
    
#     elif bank == "icici":
#         mapping = {
#             "Transaction Date": "Date",
#             "Transaction Remarks": "Description",
#             "Withdrawal Amount(INR )": "Debit",
#             "Deposit Amount(INR )": "Credit",
#             "Balance (INR )": "Balance"
#         }
    
#     elif bank == "union bank":
#         mapping = {
#             "Date": "Date",
#             "Remarks": "Description",
#             "Withdrawals": "Debit",
#             "Deposits": "Credit",
#             "Balance": "Balance"
#         }
    
#     elif bank == "sbi":
#         mapping = {
#             "Txn Date": "Date",
#             "Description": "Description",
#             "Debit": "Debit",
#             "Credit": "Credit",
#             "Balance": "Balance"
#         }
    
#     else:
#         raise ValueError(f"Bank '{bank}' not supported")
    
#     # Apply mapping
#     df_normalized = df.rename(columns=mapping)
    
#     # Keep only unified schema
#     df_normalized = df_normalized[["Date", "Description", "Debit", "Credit", "Balance"]]
    
#     return df_normalized


In [66]:
import pandas as pd

def normalize_bank_statement(df: pd.DataFrame, bank: str) -> pd.DataFrame:
    bank = bank.lower().strip()
    
    if bank == "hdfc":
        mapping = {
            "Date": "Date",
            "Narration": "Description",
            "Withdrawal": "Debit",
            "Deposit": "Credit",
            "Closing Balance": "Balance"
        }

    elif bank == "icici":
        mapping = {
            "Value Date": "Date",          # sometimes "Value Date" also exists
            "Transaction Remarks": "Description",
            "Withdrawal Amount\n(INR )": "Debit",  # fixed spacing
            "Deposit Amount\n(INR )": "Credit",    # fixed spacing
            "Balance (INR )": "Balance"
        }
    
    elif bank == "union bank":
        mapping = {
            "Date": "Date",
            "Remarks": "Description",
            "Withdrawals": "Debit",
            "Deposits": "Credit",
            "Balance": "Balance"
        }
    
    elif bank == "sbi":
        mapping = {
            "Txn Date": "Date",
            "Description": "Description",
            "Debit": "Debit",
            "Credit": "Credit",
            "Balance": "Balance"
        }
    
    else:
        raise ValueError(f"Bank '{bank}' not supported")
    
    # Strip column names to avoid mismatch due to spaces
    df.columns = df.columns.str.strip()
    
    # Apply mapping safely (only map existing columns)
    df_normalized = df.rename(columns=mapping)
    
    # Ensure all required columns exist, even if missing in original
    for col in ["Date", "Description", "Debit", "Credit", "Balance"]:
        if col not in df_normalized.columns:
            df_normalized[col] = None
    
    # Keep only unified schema
    df_normalized = df_normalized[["Date", "Description", "Debit", "Credit", "Balance"]]
    
    return df_normalized


## Process the Normalization

In [60]:
def process_pdf(pdf_path: str, bank: str) -> pd.DataFrame:
    raw_df = extract_from_pdf(pdf_path)
    normalized_df = normalize_bank_statement(raw_df, bank)
    return normalized_df

## Example Inference

### Union Bank

In [67]:
pdf_path = "/Users/karthiksagar/Expense-Classification/Statements/ubi1.pdf"
bank = "Union Bank"  # or "ICICI", "Union Bank", "SBI"

ubi = process_pdf(pdf_path, bank)
ubi.head()

,Date,Description,Debit,Credit,Balance
0,03-04-2024\n14:42:12,IMPSAB/40941472988\n7/Narasimha Rao\nNagul/703...,,"2,00,000.00","-38,37,580.09"
1,03-04-2024\n15:38:34,IMPSAB/40941573799\n6/Narasimha Rao\nNagul/703...,,"1,00,000.00","-37,37,580.09"
2,03-04-2024\n16:02:03,IMPSAB/40941674267\n4/Narasimha Rao\nNagul/703...,,"3,000.00","-37,34,580.09"
3,04-04-2024\n04:49:35,TR VL 165544,"31,000.00",,"-37,65,580.09"
4,06-04-2024\n16:09:21,IMPSAB/40971619091\n5/Narasimha Rao\nNagul/703...,,"2,00,000.00","-35,65,580.09"


### SBI

In [68]:
pdf_path = "/Users/karthiksagar/Expense-Classification/Statements/sbi1.pdf"
bank = "SBI"  # or "ICICI", "Union Bank", "SBI"

sbi = process_pdf(pdf_path, bank)
sbi.head()

,Date,Description,Debit,Credit,Balance
0,23 May2025,TO TRANSFER-UPI/DR/343609514600/DEVENDER/PPIW/...,"1,000.00",,"2,08,846.95"
1,23 May2025,TO TRANSFER-UPI/DR/783375161387/THIRUMAL/YESB/...,"4,000.00",,"2,04,846.95"
2,23 May2025,TO TRANSFER-UPI/DR/514386563335/LETAKARI/YESB/...,500.00,,"2,04,346.95"
3,23 May2025,TO TRANSFER-UPI/DR/514386601258/NALLAGUL/UBIN/...,"2,000.00",,"2,02,346.95"
4,25 May2025,TO TRANSFER-UPI/DR/514585928702/PRODDATU/FDRL/...,"1,000.00",,"2,01,346.95"


### ICICI

In [ ]:
pdf_path = "/Users/karthiksagar/Expense-Classification/Statements/icici.pdf"
bank = "ICICI"  # or "ICICI", "Union Bank", "SBI"

df = process_pdf(pdf_path, bank)
df.head()

KeyError: "None of [Index(['Date', 'Description', 'Debit', 'Credit', 'Balance'], dtype='object')] are in the [columns]"

### HDFC

In [69]:
pdf_path = "/Users/karthiksagar/Expense-Classification/Statements/hdfc1.pdf"
bank = "hdfc"  # or "ICICI", "Union Bank", "SBI"

hdfc = process_pdf(pdf_path, bank)
hdfc.head()

,Date,Description,Debit,Credit,Balance
0,01 Feb 2024,IMPS-403203440352-ZERODHABROKING LTD--HDFC-xxx...,,0.01,None
1,31 Jan 2024,UPI-Apple Services-appleservices.bdsi@icici-IC...,499.00,,None
2,29 Jan 2024,UPI-HEMANTH KUMAR K R-hemanthsiva2022@oksbi-SB...,,100.00,None
3,28 Jan 2024,UPI-M PAUL JONAS JAI DEE-pauljonas1818@okicici...,,200.00,None
4,28 Jan 2024,UPI-NALLAGULA PADMAJA-karthik.sagarn-1@okaxis-...,,"1,000.00",None


In [72]:
combine = pd.concat(objs=[sbi, ubi, hdfc])

In [73]:
combine

,Date,Description,Debit,Credit,Balance
0,23 May2025,TO TRANSFER-UPI/DR/343609514600/DEVENDER/PPIW/...,"1,000.00",,"2,08,846.95"
1,23 May2025,TO TRANSFER-UPI/DR/783375161387/THIRUMAL/YESB/...,"4,000.00",,"2,04,846.95"
2,23 May2025,TO TRANSFER-UPI/DR/514386563335/LETAKARI/YESB/...,500.00,,"2,04,346.95"
3,23 May2025,TO TRANSFER-UPI/DR/514386601258/NALLAGUL/UBIN/...,"2,000.00",,"2,02,346.95"
4,25 May2025,TO TRANSFER-UPI/DR/514585928702/PRODDATU/FDRL/...,"1,000.00",,"2,01,346.95"
...,...,...,...,...,...
6,27 Jan 2024,UPI-Tarun Sakthivel-tarunscbe@oksbi-SBIN001547...,,85.00,None
7,27 Jan 2024,UPI-Boggarapu Manikanta -manikantasaiabhinay20...,250.00,,None
8,26 Jan 2024,UPI-KOMMURI BHUPATHI-q281785269@ybl-YESB0YBLUP...,10.00,,None
9,25 Jan 2024,UPI-DEVENDER SAGAR N-nds.august21@oksbi-SBIN00...,,"2,000.00",None


In [78]:
combine.to_csv('output.csv', index=False)

### Further more cleaning

In [80]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/karthiksagar/Expense-Classification/output.csv')

# Clean the 'Date' column
# The data in the 'Date' column seems to have a date and time separated by a newline character ('\n').
# We can split the string by the newline character and take the first part, which is the date.
df['Date'] = df['Date'].str.split('\n').str[0]

# Convert the 'Date' column to datetime objects
# This step is good practice to ensure the dates are in a consistent format.
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_output.csv', index=False)

# Display the first few rows of the cleaned data to verify the changes
print("Cleaned DataFrame:")
print(df.head())

ValueError: time data "03-04-2024" doesn't match format "%d %B%Y", at position 3. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [83]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('/Users/karthiksagar/Expense-Classification/output.csv')

# Clean the 'Date' column by removing the time part
df['Date'] = df['Date'].str.split('\n').str[0]

# Convert the 'Date' column to datetime objects, handling mixed formats
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True)

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_output.csv', index=False)

df

,Date,Description,Debit,Credit,Balance
0,2025-05-23,TO TRANSFER-UPI/DR/343609514600/DEVENDER/PPIW/...,"1,000.00",NaN,"2,08,846.95"
1,2025-05-23,TO TRANSFER-UPI/DR/783375161387/THIRUMAL/YESB/...,"4,000.00",NaN,"2,04,846.95"
2,2025-05-23,TO TRANSFER-UPI/DR/514386563335/LETAKARI/YESB/...,500.00,NaN,"2,04,346.95"
3,2025-05-23,TO TRANSFER-UPI/DR/514386601258/NALLAGUL/UBIN/...,"2,000.00",NaN,"2,02,346.95"
4,2025-05-25,TO TRANSFER-UPI/DR/514585928702/PRODDATU/FDRL/...,"1,000.00",NaN,"2,01,346.95"
...,...,...,...,...,...
324,2024-01-27,UPI-Tarun Sakthivel-tarunscbe@oksbi-SBIN001547...,NaN,85.00,NaN
325,2024-01-27,UPI-Boggarapu Manikanta -manikantasaiabhinay20...,250.00,NaN,NaN
326,2024-01-26,UPI-KOMMURI BHUPATHI-q281785269@ybl-YESB0YBLUP...,10.00,NaN,NaN
327,2024-01-25,UPI-DEVENDER SAGAR N-nds.august21@oksbi-SBIN00...,NaN,"2,000.00",NaN


In [ ]:
df.to_csv()